In [43]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [45]:
# Training settings
#parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
#parser.add_argument('--batch-size', type=int, default=64, metavar='N',
#        help='input batch size for training (default: 64)')
#parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
#        help='input batch size for testing (default: 1000)')
#parser.add_argument('--epochs', type=int, default=10, metavar='N',
#        help='number of epochs to train (default: 10)')
#parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
#        help='learning rate (default: 0.01)')
#parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
#        help='SGD momentum (default: 0.5)')
#parser.add_argument('--no-cuda', action='store_true', default=False,
#        help='disables CUDA training')
#parser.add_argument('--seed', type=int, default=1, metavar='S',
#        help='random seed (default: 1)')
#parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#        help='how many batches to wait before logging training status')
    
#parser.add_argument('--save-model', action='store_true', default=False,
#        help='For Saving the current Model')
#args = parser.parse_args()
#use_cuda = not args.no_cuda and torch.cuda.is_available()

#torch.manual_seed(args.seed)

device = torch.device("cpu")

#kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/Data/MNIST', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('~/Data/MNIST', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])), batch_size=1000, shuffle=True)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

#for epoch in range(1, 10 + 1):
train(model, device, train_loader, optimizer, epoch)
test(model, device, test_loader)

#torch.save(model.state_dict(),"mnist_cnn.pt")

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.309495
Train Epoch: 3 [64/60000 (0%)]	Loss: 2.316519
Train Epoch: 3 [128/60000 (0%)]	Loss: 2.318918
Train Epoch: 3 [192/60000 (0%)]	Loss: 2.297460
Train Epoch: 3 [256/60000 (0%)]	Loss: 2.299594
Train Epoch: 3 [320/60000 (1%)]	Loss: 2.289636
Train Epoch: 3 [384/60000 (1%)]	Loss: 2.280693
Train Epoch: 3 [448/60000 (1%)]	Loss: 2.269134
Train Epoch: 3 [512/60000 (1%)]	Loss: 2.269644
Train Epoch: 3 [576/60000 (1%)]	Loss: 2.255248
Train Epoch: 3 [640/60000 (1%)]	Loss: 2.252802
Train Epoch: 3 [704/60000 (1%)]	Loss: 2.240483
Train Epoch: 3 [768/60000 (1%)]	Loss: 2.249603
Train Epoch: 3 [832/60000 (1%)]	Loss: 2.224535
Train Epoch: 3 [896/60000 (1%)]	Loss: 2.223684
Train Epoch: 3 [960/60000 (2%)]	Loss: 2.227518
Train Epoch: 3 [1024/60000 (2%)]	Loss: 2.209362
Train Epoch: 3 [1088/60000 (2%)]	Loss: 2.216445
Train Epoch: 3 [1152/60000 (2%)]	Loss: 2.204900
Train Epoch: 3 [1216/60000 (2%)]	Loss: 2.205486
Train Epoch: 3 [1280/60000 (2%)]	Loss: 2.172009
Train Epoch

Train Epoch: 3 [10944/60000 (18%)]	Loss: 0.368704
Train Epoch: 3 [11008/60000 (18%)]	Loss: 0.485634
Train Epoch: 3 [11072/60000 (18%)]	Loss: 0.452424
Train Epoch: 3 [11136/60000 (19%)]	Loss: 0.497470
Train Epoch: 3 [11200/60000 (19%)]	Loss: 0.452016
Train Epoch: 3 [11264/60000 (19%)]	Loss: 0.278255
Train Epoch: 3 [11328/60000 (19%)]	Loss: 0.419029
Train Epoch: 3 [11392/60000 (19%)]	Loss: 0.226677
Train Epoch: 3 [11456/60000 (19%)]	Loss: 0.329326
Train Epoch: 3 [11520/60000 (19%)]	Loss: 0.337544
Train Epoch: 3 [11584/60000 (19%)]	Loss: 0.351987
Train Epoch: 3 [11648/60000 (19%)]	Loss: 0.307514
Train Epoch: 3 [11712/60000 (20%)]	Loss: 0.439190
Train Epoch: 3 [11776/60000 (20%)]	Loss: 0.325761
Train Epoch: 3 [11840/60000 (20%)]	Loss: 0.261858
Train Epoch: 3 [11904/60000 (20%)]	Loss: 0.371715
Train Epoch: 3 [11968/60000 (20%)]	Loss: 0.183077
Train Epoch: 3 [12032/60000 (20%)]	Loss: 0.180953
Train Epoch: 3 [12096/60000 (20%)]	Loss: 0.236775
Train Epoch: 3 [12160/60000 (20%)]	Loss: 0.139451


Train Epoch: 3 [21568/60000 (36%)]	Loss: 0.134270
Train Epoch: 3 [21632/60000 (36%)]	Loss: 0.146935
Train Epoch: 3 [21696/60000 (36%)]	Loss: 0.225163
Train Epoch: 3 [21760/60000 (36%)]	Loss: 0.352751
Train Epoch: 3 [21824/60000 (36%)]	Loss: 0.291257
Train Epoch: 3 [21888/60000 (36%)]	Loss: 0.195500
Train Epoch: 3 [21952/60000 (37%)]	Loss: 0.139480
Train Epoch: 3 [22016/60000 (37%)]	Loss: 0.157562
Train Epoch: 3 [22080/60000 (37%)]	Loss: 0.207309
Train Epoch: 3 [22144/60000 (37%)]	Loss: 0.289192
Train Epoch: 3 [22208/60000 (37%)]	Loss: 0.111238
Train Epoch: 3 [22272/60000 (37%)]	Loss: 0.362078
Train Epoch: 3 [22336/60000 (37%)]	Loss: 0.142525
Train Epoch: 3 [22400/60000 (37%)]	Loss: 0.255209
Train Epoch: 3 [22464/60000 (37%)]	Loss: 0.195132
Train Epoch: 3 [22528/60000 (38%)]	Loss: 0.223602
Train Epoch: 3 [22592/60000 (38%)]	Loss: 0.253584
Train Epoch: 3 [22656/60000 (38%)]	Loss: 0.127318
Train Epoch: 3 [22720/60000 (38%)]	Loss: 0.304107
Train Epoch: 3 [22784/60000 (38%)]	Loss: 0.178194


Train Epoch: 3 [32192/60000 (54%)]	Loss: 0.129210
Train Epoch: 3 [32256/60000 (54%)]	Loss: 0.172979
Train Epoch: 3 [32320/60000 (54%)]	Loss: 0.174896
Train Epoch: 3 [32384/60000 (54%)]	Loss: 0.065265
Train Epoch: 3 [32448/60000 (54%)]	Loss: 0.301899
Train Epoch: 3 [32512/60000 (54%)]	Loss: 0.160505
Train Epoch: 3 [32576/60000 (54%)]	Loss: 0.089452
Train Epoch: 3 [32640/60000 (54%)]	Loss: 0.148979
Train Epoch: 3 [32704/60000 (54%)]	Loss: 0.118724
Train Epoch: 3 [32768/60000 (55%)]	Loss: 0.225685
Train Epoch: 3 [32832/60000 (55%)]	Loss: 0.083680
Train Epoch: 3 [32896/60000 (55%)]	Loss: 0.190398
Train Epoch: 3 [32960/60000 (55%)]	Loss: 0.199537
Train Epoch: 3 [33024/60000 (55%)]	Loss: 0.107738
Train Epoch: 3 [33088/60000 (55%)]	Loss: 0.310905
Train Epoch: 3 [33152/60000 (55%)]	Loss: 0.233961
Train Epoch: 3 [33216/60000 (55%)]	Loss: 0.105973
Train Epoch: 3 [33280/60000 (55%)]	Loss: 0.141695
Train Epoch: 3 [33344/60000 (56%)]	Loss: 0.090306
Train Epoch: 3 [33408/60000 (56%)]	Loss: 0.079425


Train Epoch: 3 [42688/60000 (71%)]	Loss: 0.151134
Train Epoch: 3 [42752/60000 (71%)]	Loss: 0.094115
Train Epoch: 3 [42816/60000 (71%)]	Loss: 0.150639
Train Epoch: 3 [42880/60000 (71%)]	Loss: 0.102069
Train Epoch: 3 [42944/60000 (72%)]	Loss: 0.077542
Train Epoch: 3 [43008/60000 (72%)]	Loss: 0.106308
Train Epoch: 3 [43072/60000 (72%)]	Loss: 0.215041
Train Epoch: 3 [43136/60000 (72%)]	Loss: 0.135466
Train Epoch: 3 [43200/60000 (72%)]	Loss: 0.154002
Train Epoch: 3 [43264/60000 (72%)]	Loss: 0.120897
Train Epoch: 3 [43328/60000 (72%)]	Loss: 0.239472
Train Epoch: 3 [43392/60000 (72%)]	Loss: 0.177389
Train Epoch: 3 [43456/60000 (72%)]	Loss: 0.174784
Train Epoch: 3 [43520/60000 (72%)]	Loss: 0.077886
Train Epoch: 3 [43584/60000 (73%)]	Loss: 0.158259
Train Epoch: 3 [43648/60000 (73%)]	Loss: 0.154894
Train Epoch: 3 [43712/60000 (73%)]	Loss: 0.049923
Train Epoch: 3 [43776/60000 (73%)]	Loss: 0.211473
Train Epoch: 3 [43840/60000 (73%)]	Loss: 0.084251
Train Epoch: 3 [43904/60000 (73%)]	Loss: 0.106204


Train Epoch: 3 [53376/60000 (89%)]	Loss: 0.079119
Train Epoch: 3 [53440/60000 (89%)]	Loss: 0.115678
Train Epoch: 3 [53504/60000 (89%)]	Loss: 0.262982
Train Epoch: 3 [53568/60000 (89%)]	Loss: 0.072413
Train Epoch: 3 [53632/60000 (89%)]	Loss: 0.088486
Train Epoch: 3 [53696/60000 (89%)]	Loss: 0.285711
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.078637
Train Epoch: 3 [53824/60000 (90%)]	Loss: 0.071374
Train Epoch: 3 [53888/60000 (90%)]	Loss: 0.073297
Train Epoch: 3 [53952/60000 (90%)]	Loss: 0.125762
Train Epoch: 3 [54016/60000 (90%)]	Loss: 0.124513
Train Epoch: 3 [54080/60000 (90%)]	Loss: 0.071874
Train Epoch: 3 [54144/60000 (90%)]	Loss: 0.134433
Train Epoch: 3 [54208/60000 (90%)]	Loss: 0.165075
Train Epoch: 3 [54272/60000 (90%)]	Loss: 0.126204
Train Epoch: 3 [54336/60000 (91%)]	Loss: 0.196861
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.138040
Train Epoch: 3 [54464/60000 (91%)]	Loss: 0.122463
Train Epoch: 3 [54528/60000 (91%)]	Loss: 0.038132
Train Epoch: 3 [54592/60000 (91%)]	Loss: 0.205832
